In [1]:
import numpy as np
import sympy

from optlang import Model, Variable, Constraint, Objective

# All the (symbolic) variables are declared, with a name and optionally a lower and/or upper bound.
x1 = Variable('x1', lb=5, ub=100, type="integer")
x2 = Variable('x2', lb=0, ub=100, type="integer")
x3 = Variable('x3', lb=0, ub=100, type="integer")

# A constraint is constructed from an expression of variables and a lower and/or upper bound (lb and ub).
c1 = Constraint(x1 + x2 + x3, ub=100)
c2 = Constraint(10 * x1 + 4 * x2 + 5 * x3, ub=600)
c3 = Constraint(2 * x1 + 2 * x2 + 6 * x3, ub=300)

# An objective can be formulated
obj = Objective(10 * x1 + 6 * x2 + 4 * x3, direction='max')

# Variables, constraints and objective are combined in a Model object, which can subsequently be optimized.
prob = Model(name='Simple model')
prob.objective = obj
prob.add([c1, c2, c3])

status = prob.optimize()

opt_sol = {}
print("status:", prob.status)
print("objective value:", prob.objective.value)
print("----------")
for var_name, var in prob.variables.iteritems():
    print(var_name, "=", var.primal)
    opt_sol[var] = var.primal

status: optimal
objective value: 732.0
----------
x1 = 33.0
x2 = 67.0
x3 = 0.0


In [2]:
from neurop.problem import ILPProblem
from neurop.compiler import ILPQUBOCompiler
from neurop.model import QUBOModel
from neurop.backend import SimulatedAnnealingBackend

problem = ILPProblem(prob)
backend = SimulatedAnnealingBackend(np.logspace(5, -1, 100_000))
converter = ILPQUBOCompiler(problem, backend=backend, penalty=5)
model = converter.compile()


In [3]:
sol_energy, sol_assignment = backend.run(model)
problem_params = converter.model_to_problem_parameters(sol_assignment)
problem_params

Temperature: 0.1  Current energy: -4246426: 100%|██████████| 100000/100000 [00:05<00:00, 17107.93it/s]               


{5 <= x1 <= 100: 82, 0 <= x2 <= 100: 66, 0 <= x3 <= 100: 10}

In [4]:
problem.evaluate_objective(problem_params)

1256.00000000000

In [5]:
problem.evaluate_constraints(problem_params)

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True, False],
       [ True, False],
       [ True, False]])

In [6]:
sympy.Matrix(model.Q)

Matrix([
[-124360,    2120,    4240,    8480,    16960,    33920,    67840,    450,     900,    1800,    3600,    7200,    14400,    28800,    630,    1260,    2520,    5040,    10080,    20160,    40320,    20,     40,     80,    160,    320,     640,    1280,    2560,    5120,    100,    200,    400,    800,    1600,    3200,    6400,    12800,    25600,    51200,    10,    20,    40,     80,    160,    320,    640,     0,     0,     0,      0,      0,      0,      0,     0,     0,     0,      0,      0,      0,      0,    10,    20,    40,     80,    160,    320,    640],
[   2120, -246600,    8480,   16960,    33920,    67840,   135680,    900,    1800,    3600,    7200,   14400,    28800,    57600,   1260,    2520,    5040,   10080,    20160,    40320,    80640,    40,     80,    160,    320,    640,    1280,    2560,    5120,   10240,    200,    400,    800,   1600,    3200,    6400,   12800,    25600,    51200,   102400,    20,    40,    80,    160,    320,    640,   1280,     0

In [7]:
var_a = sympy.Array(model.variables, (len(model.variables),1))
(var_a.transpose() @ sympy.Matrix(model.Q) @ var_a)[0].expand()

-5790*slack_0_0**2 + 40*slack_0_0*slack_0_1 + 80*slack_0_0*slack_0_2 + 160*slack_0_0*slack_0_3 + 320*slack_0_0*slack_0_4 + 640*slack_0_0*slack_0_5 + 1280*slack_0_0*slack_0_6 + 2560*slack_0_0*slack_0_7 + 5120*slack_0_0*slack_0_8 + 40*slack_0_0*x1_0 + 80*slack_0_0*x1_1 + 160*slack_0_0*x1_2 + 320*slack_0_0*x1_3 + 640*slack_0_0*x1_4 + 1280*slack_0_0*x1_5 + 2560*slack_0_0*x1_6 + 40*slack_0_0*x2_0 + 80*slack_0_0*x2_1 + 160*slack_0_0*x2_2 + 320*slack_0_0*x2_3 + 640*slack_0_0*x2_4 + 1280*slack_0_0*x2_5 + 2560*slack_0_0*x2_6 + 120*slack_0_0*x3_0 + 240*slack_0_0*x3_1 + 480*slack_0_0*x3_2 + 960*slack_0_0*x3_3 + 1920*slack_0_0*x3_4 + 3840*slack_0_0*x3_5 + 7680*slack_0_0*x3_6 - 11560*slack_0_1**2 + 160*slack_0_1*slack_0_2 + 320*slack_0_1*slack_0_3 + 640*slack_0_1*slack_0_4 + 1280*slack_0_1*slack_0_5 + 2560*slack_0_1*slack_0_6 + 5120*slack_0_1*slack_0_7 + 10240*slack_0_1*slack_0_8 + 80*slack_0_1*x1_0 + 160*slack_0_1*x1_1 + 320*slack_0_1*x1_2 + 640*slack_0_1*x1_3 + 1280*slack_0_1*x1_4 + 2560*slack_0_

In [9]:
import neal
import dimod

bqm=dimod.BinaryQuadraticModel(np.array(model.Q, dtype=int), "BINARY")
sa = neal.SimulatedAnnealingSampler()
sampleset = sa.sample(bqm, beta_schedule_type="geometric", num_reads=1000, num_sweeps=10000)
sampleset.relabel_variables(dict(enumerate(model.variables)))
#decoded_samples = model.decode_sampleset(sampleset)
#best_sample = min(decoded_samples, key=lambda x: x.energy)
#pprint(best_sample.sample)
sol_assignment_neal=sampleset.lowest().first.sample

problem_params_neal = converter.model_to_problem_parameters(sol_assignment_neal)
problem_params_neal

{5 <= x1 <= 100: 36, 0 <= x2 <= 100: 59, 0 <= x3 <= 100: 1}

In [10]:
problem.evaluate_objective(problem_params_neal)

718.000000000000

In [11]:
problem.evaluate_constraints(problem_params_neal)

array([[ True,  True],
       [ True,  True],
       [ True,  True],
       [ True,  True],
       [ True, False],
       [ True,  True]])

In [12]:
print("==== Results ====")
for k in problem.problem.variables:
    print("{}: ours: {} neal: {} optimal: {}".format(k.name, problem_params[k], problem_params_neal[k], opt_sol[k]))

==== Results ====
x1: ours: 82 neal: 36 optimal: 33.0
x2: ours: 66 neal: 59 optimal: 67.0
x3: ours: 10 neal: 1 optimal: 0.0


In [12]:
np.save("q_mat.npy", model.Q)